In [32]:
from keras.layers import Input, Lambda, Dense, Flatten
from keras.models import Model
from tensorflow.keras.applications.resnet50 import ResNet50
from tensorflow.keras.applications.resnet50 import preprocess_input
from keras.preprocessing import image
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
import numpy as np
from glob import glob
import matplotlib.pyplot as plt

In [12]:
IMAGE_SIZE = [224, 224]

In [13]:
train_path = 'Datasets/Train'
valid_path = 'Datasets/Test'

In [15]:
res = ResNet50(input_shape=IMAGE_SIZE + [3], weights='imagenet', include_top=False)

94765736/94765736 [==============================] - 366s 4us/step


In [18]:
for layer in res.layers:
  layer.trainable = False

In [19]:
folders = glob('Datasets/Train/*')

In [20]:
x = Flatten()(res.output)

In [21]:
prediction = Dense(len(folders), activation='softmax')(x)

In [23]:
model = Model(inputs=res.input, outputs=prediction)

# view the structure of the model
model.summary()

Model: "model"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_1 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 conv1_pad (ZeroPadding2D)      (None, 230, 230, 3)  0           ['input_1[0][0]']                
                                                                                                  
 conv1_conv (Conv2D)            (None, 112, 112, 64  9472        ['conv1_pad[0][0]']              
                                )                                                                 
                                                                                              

In [24]:
model.compile(
  loss='categorical_crossentropy',
  optimizer='adam',
  metrics=['accuracy']
)

In [25]:
from keras.preprocessing.image import ImageDataGenerator

train_datagen = ImageDataGenerator(rescale = 1./255,
                                   shear_range = 0.2,
                                   zoom_range = 0.2,
                                   horizontal_flip = True)

test_datagen = ImageDataGenerator(rescale = 1./255)

training_set = train_datagen.flow_from_directory('Datasets/Train',
                                                 target_size = (224, 224),
                                                 batch_size = 32,
                                                 class_mode = 'categorical')

test_set = test_datagen.flow_from_directory('Datasets/Test',
                                            target_size = (224, 224),
                                            batch_size = 32,
                                            class_mode = 'categorical')

Found 1399 images belonging to 4 classes.
Found 6869 images belonging to 4 classes.


In [31]:
# fit the model
r = model.fit_generator(
  training_set,
  validation_data=test_set,
  epochs=5,
  steps_per_epoch=len(training_set),
  validation_steps=len(test_set)
)

C:\Users\HP\AppData\Local\Temp/ipykernel_2648/708679198.py:2: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  r = model.fit_generator(


Epoch 1/5
44/44 [==============================] - 1025s 24s/step - loss: 2.0438 - accuracy: 0.3288 - val_loss: 2.2645 - val_accuracy: 0.2447
Epoch 2/5
44/44 [==============================] - 1034s 24s/step - loss: 1.4962 - accuracy: 0.3996 - val_loss: 2.2630 - val_accuracy: 0.2273
Epoch 3/5
44/44 [==============================] - 1064s 25s/step - loss: 1.3148 - accuracy: 0.4246 - val_loss: 2.4225 - val_accuracy: 0.2255
Epoch 4/5
44/44 [==============================] - 1056s 24s/step - loss: 1.5724 - accuracy: 0.3946 - val_loss: 2.2494 - val_accuracy: 0.2418
Epoch 5/5
44/44 [==============================] - 1196s 28s/step - loss: 1.5614 - accuracy: 0.4010 - val_loss: 2.5209 - val_accuracy: 0.2487


In [33]:
model.save("my_model.h5")

In [34]:
from tensorflow import keras
model = keras.models.load_model(r'C:\Users\HP\VSCode Projects\AIproject\Transfer-Learning\my_model.h5')
model

In [29]:
model.predict(training_set)

44/44 [==============================] - 187s 4s/step


array([[0.00157598, 0.36220807, 0.60072136, 0.03549457],
       [0.01838448, 0.1488019 , 0.5893089 , 0.24350469],
       [0.03620238, 0.16099507, 0.42945182, 0.37335074],
       ...,
       [0.04142052, 0.32528436, 0.30400154, 0.32929358],
       [0.02738629, 0.11118669, 0.7449016 , 0.11652537],
       [0.02630988, 0.14068744, 0.63109064, 0.20191212]], dtype=float32)

In [36]:
model.fit(test_set)

215/215 [==============================] - 991s 5s/step - loss: 1.6639 - accuracy: 0.3644
